In [ ]:
# | default_exp _cli_code_generator

In [ ]:
# | export

from typing import *

import typer

from fastkafka._components.logger import get_logger
from fastkafka._code_generator.app_description_validator import validate_app_description
from fastkafka._code_generator.plan_generator import generate_plan
from fastkafka._code_generator.app_generator import generate_app
from fastkafka._code_generator.test_generator import generate_test

In [ ]:
from typer.testing import CliRunner

from fastkafka._components.logger import suppress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
runner = CliRunner()

In [ ]:
# | export

_code_generator_app = typer.Typer(help="Commands for accelerating FastKafka app creation using advanced AI technology")

In [ ]:
# | export


@_code_generator_app.command(
    "generate",
    help="Generate a new FastKafka app(s) effortlessly with advanced AI assistance",
)
def generate_fastkafka_app(
    description: str = typer.Argument(
        ...,
        help="""Summarize your FastKafka app in a few sentences!


\nInclude details about message classes, FastKafka app configuration (e.g., kafka_brokers), consumer and producer functions, and specify the business logic to be implemented. 


\nThe simpler and more specific the app description is, the better the generated app will be. Please refer to the below example for inspiration:


\nCreate a FastKafka application that consumes messages from the "store_product" topic. These messages should have three attributes: "product_name," "currency," and "price". While consuming, the app needs to produce a message to the "change_currency" topic. The function responsible for producing should take a "store_product" object as input and return the same object. Additionally, this function should check if the currency in the input "store_product" is "HRK." If it is, then the currency should be changed to "EUR," and the price should be divided by 7.5. Remember, the app should use a "localhost" broker.


\n"""
    )
) -> None:
    """Generate a new FastKafka app(s) effortlessly with advanced AI assistance"""
    try:
        validate_app_description(description)
        plan = generate_plan(description)
        code = generate_app(plan)
        test = generate_test(code)
        
        typer.secho(" ▶ Total tokens usage: ~32.9k", fg=typer.colors.CYAN)        
        typer.secho("✨  All files were successfully generated.!", fg=typer.colors.CYAN)
    except Exception as e:
        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)

In [ ]:
# | notest

! nbdev_export

In [ ]:
result = runner.invoke(_code_generator_app, ["generate", "--help"])

                                                                                                                   
 Usage: generate [OPTIONS] DESCRIPTION                                                                             
                                                                                                                   

Generate a new FastKafka app(s) effortlessly with advanced AI assistance

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    description      TEXT  Summarize your FastKafka app in a few sentences!                                    │
│                                                                                                                 │
│                             Include details about message classes, FastKafka app configuration (e.g.,           │
│                             kafka_brokers), consumer and producer functions, and specify the business logic to  │
│                             be implemented.                                                                     │
│                                                                                                                 │
│                             The simpler and more specific the app description is, the better the generated app  │
│                             will be. Please refer to the below example for inspiration:                         │
│                                                                                                                 │
│                             Create a FastKafka application that consumes messages from the "store_product"      │
│                             topic. These messages should have three attributes: "product_name," "currency," and │
│                             "price". While consuming, the app needs to produce a message to the                 │
│                             "change_currency" topic. The function responsible for producing should take a       │
│                             "store_product" object as input and return the same object. Additionally, this      │
│                             function should check if the currency in the input "store_product" is "HRK." If it  │
│                             is, then the currency should be changed to "EUR," and the price should be divided   │
│                             by 7.5. Remember, the app should use a "localhost" broker.                          │
│                             [default: None]                                                                     │
│                             [required]                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --install-completion          Install completion for the current shell.                                         │
│ --show-completion             Show completion for the current shell, to copy it or customize the installation.  │
│ --help                        Show this message and exit.                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# result = runner.invoke(_code_generator_app, ["Sample FastKafka application description"])
# assert result.exit_code == 0, result.exit_code